In [35]:
import jsonlines
import numpy as np
import pandas as pd
from process_data import DataProcessor
from nltk.tokenize import word_tokenize, wordpunct_tokenize, sent_tokenize
from pathlib import Path
from sklearn.feature_extraction.text import TfidfVectorizer

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [26]:
INPUT_PATH = "./example-data/comments"
OUTPUT_PATH = "./example-output"
MODE = "comments"
MAX_DOCS = 500
MIN_TOKENS = 50
MAX_TOKENS = 100
PHRASES = ["this community", "this subreddit", "this place", "you guys", "you all"]

In [27]:
p = DataProcessor(
        input_path=INPUT_PATH,
        output_directory=OUTPUT_PATH,
        mode=MODE,
    )
gen = p.get_generator()
count = 0
n = 0
docs = []

for line in gen:
    n += 1
    text = line.get("body")
    if text:
        if line.get("author") == "AutoModerator":
            pass
        else:
            for phrase in PHRASES:
                if phrase in text.lower():
                    l = len(wordpunct_tokenize(text))
                    if l >= MIN_TOKENS and l <= MAX_TOKENS:
                        docs.append(text)
                        count += 1
                if count >= MAX_DOCS:
                    raise StopIteration(f"Iterated over {n} lines.")

StopIteration: Iterated over 546578 lines.

In [29]:
# get array

vectorizer = TfidfVectorizer(max_df=.65, min_df=1, stop_words=None, use_idf=True, norm=None)
docs_array = vectorizer.fit_transform(docs).toarray()

In [30]:
docs_array.shape

(500, 4223)

In [31]:
# get the list of tokens

columns = list(vectorizer.get_feature_names_out()) 

In [32]:
doc_dfs = []
for i, doc_array in enumerate(docs_array):
    # construct a dataframe
    tf_idf_tuples = list(zip(columns, doc_array))
    df = pd.DataFrame.from_records(tf_idf_tuples, columns=['term', 'score']).sort_values(by='score', ascending=False).reset_index(drop=True)
    doc_dfs.append(df)

In [33]:
# get top N keywords of every comment

TOP_N = 5
docs_dict = {}
for i, df in enumerate(doc_dfs):
    sdf = df.iloc[:TOP_N]
    terms = list(sdf["term"])
    docs_dict[i] = terms

top_df = pd.DataFrame(docs_dict)

In [36]:
top_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499
0,white,white,shit,media,europe,sorry,supress,mistakes,brutality,screenshot,arse,harry,choice,take,rent,choose,exhausted,into,happens,connection,private,photoshopped,aren,delivering,pleasure,about,pair,any,half,innocent,id,object,awkward,their,issue,black,rice,language,garbage,ll,tuition,may,men,onlyfans,family,disorders,delicately,dont,dying,restaurant,waste,men,paper,distance,hands,bad,hand,love,weak,lost,century,sex,happening,this,mortgage,houses,funny,her,water,them,them,who,him,anal,covid,vaccine,partner,your,carrier,territory,prob,sick,top,lie,eat,maybe,naturally,wrong,her,ya,peeping,which,800k,tomorrows,india,conversation,feel,yippy,yippy,child,sure,pics,her,say,am,number,each,lvl,speech,help,tall,true,ups,im,from,never,gender,he,once,countries,greater,broccoli,advice,her,news,muslim,repeated,same,longer,gfs,condom,difference,live,myself,side,her,their,gives,uncomfortable,guy,ignorant,50,sometimes,dad,girlfriends,infection,morally,learn,divorce,depression,prescription,child,would,different,respect,take,wash,last,show,25,humbly,look,look,someone,assumptions,family,menopause,arguing,conditions,people,even,had,he,he,they,parents,treat,blah,perimenopause,same,racism,extra,parents,gay,anyone,supporting,annoying,haters,affecting,arguing,circumcised,play,eachother,doing,gorgeous,drunk,10,kg,conscious,little,instinct,negative,prices,tax,only,very,exist,exist,relationsship,pads,creepy,bond,weight,stains,straight,becomes,more,we,body,universal,own,sex,fuck,sex,unvaccinated,he,best,were,rape,forward,conditioning,shirts,leave,we,problem,know,wouldn,model,white,smile,cool,by,trying,vaxx,deer,gf,unsolicited,homophobia,hate,road,flag,sweden,us,drivers,understand,dubai,harley,was,sound,leg,mindset,where,picnic,offends,weight,fat,guess,paid,overanalysing,sarcastic,likely,meltdown,pregnancy,brings,yields,ad,why,short,cheers,seen,dumb,people,extremely,but,plays,xd,wife,integrity,ask,activity,voted,know,pastry,she,ran,checking,move,refuse,area,woman,daughter,traumatise,resist,matter,question,added,why,raving,uyghur,size,voices,our,accustomed,yes,shoes,skin,covid,passport,gift,getting,e